## Generate plots included in RSI Poster

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..

In [ ]:
import os
import pickle
from pprint import pprint

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandapower as pp

from network_utils import (
    create_56bus,
    create_RX_from_net,
    read_load_data)

In [ ]:
plots_dir = 'plots/rsi_poster/'
os.makedirs(plots_dir, exist_ok=True)

def savefig(fig: plt.Figure, filename: str) -> None:
    path = os.path.join(plots_dir, filename)
    fig.savefig(path, dpi=300, pad_inches=0, bbox_inches='tight')

### Draw SCE 56-bus network, with color-coded nodes

In [ ]:
net = create_56bus()
G_true = pp.topology.create_nxgraph(net)
R, X = create_RX_from_net(net, noise=0)  # true R and X
p, qe = read_load_data()  # in MW and MVar
T, n = p.shape

v_min, v_max = (11.4**2, 12.6**2)  # +/-5%, units kV^2
v_nom = 12**2  # nominal squared voltage magnitude, units kV^2
v_sub = v_nom  # fixed squared voltage magnitude at substation, units kV^2

vpars = qe @ X + p @ R + v_sub  # shape [T, n]
Vpar_min = np.min(vpars, axis=0)  # shape [n]
Vpar_max = np.max(vpars, axis=0)  # shape [n]

print(T, n)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 5), tight_layout=True)
pos = nx.nx_agraph.graphviz_layout(G_true, prog='dot')
nx.draw_networkx(
    G_true, pos=pos, ax=ax, node_color='white', edgecolors='black',
    node_size=150, with_labels=True, font_size=8)
ax.axis('off')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 4), tight_layout=True)

# determine node colors
node_color = ['white'] * 56
node_color[0] = 'black'
buses = [8, 18, 21, 30, 39, 45, 54]  # 0 = substation
for i, bus in enumerate(buses):
    node_color[bus] = plt.cm.tab10.colors[i]

nx.draw_networkx(
    G_true, pos=pos, ax=ax, node_color=node_color, edgecolors='black',
    node_size=100, with_labels=False)
ax.axis('off')

savefig(fig, os.path.join(plots_dir, 'sce_network.svg'))

### No control vs. perfect control

In [ ]:
vmin = 11.4  # kV
vmax = 12.6

ylim = (11.1, 13.4)

time_ticks =  [   0, 2400, 4800,  7200,  9600, 12000, 14400]
time_labels = ['0h', '4h', '8h', '12h', '16h', '20h', '24h']

buses = [18, 30]

def setup_voltage_ax(ax):
    ax.axhline(vmin, ls='--', color='black')
    ax.axhline(vmax, ls='--', color='black')
    ax.set(xlim=(-100, T), xticks=time_ticks, xticklabels=time_labels)
    ax.set(ylim=ylim, ylabel='voltage (kV)')

In [ ]:
with open('out/CBCconst_20220211_052507.pkl', 'rb') as f:
    perf_ctrl = pickle.load(f)
    print(list(perf_ctrl.keys()))

In [ ]:
figs, axs = [], []
for i in range(2):
    fig, ax = plt.subplots(1, 1, figsize=(4, 3), dpi=60, tight_layout=True)
    setup_voltage_ax(ax)
    figs.append(fig)
    axs.append(ax)

colors = ['tab:orange', 'tab:red']
for i, bus in enumerate(buses):
    axs[0].plot(np.sqrt(vpars[:, bus-1]), c=colors[i])
    axs[1].plot(np.sqrt(perf_ctrl['vs'][:, bus-1]), c=colors[i])

savefig(figs[0], path=os.path.join(plots_dir, 'no_control.svg'))
savefig(figs[1], path=os.path.join(plots_dir, 'perf_control.svg'))

### Decentralized control

In [ ]:
with open('out/decentralized_opt.pkl', 'rb') as f:
    decentral = pickle.load(f)

fig, ax = plt.subplots(1, 1, figsize=(4, 3), tight_layout=True)
setup_voltage_ax(ax)

for i, bus in enumerate(buses):
    ax.plot(np.sqrt(decentral['vs'][:, bus-1]), c=colors[i])

savefig(fig, path=os.path.join(plots_dir, 'decentral_opt.svg'))

### Wrong model

In [ ]:
with open('out/no_diag_constr/CBCconst_noise1.0_perm_norm1.0_seed8_Pu100_20220316_233154.pkl', 'rb') as f:
    wrong_model = pickle.load(f)

fig, ax = plt.subplots(1, 1, figsize=(4, 3), tight_layout=True)
setup_voltage_ax(ax)

for i, bus in enumerate(buses):
    ax.plot(np.sqrt(wrong_model['vs'][:, bus-1]), c=colors[i])

savefig(fig, path=os.path.join(plots_dir, 'wrong_model.svg'))